In [7]:
import pymongo
import mysql.connector
from pymongo import MongoClient
from bs4 import BeautifulSoup
import os
import json

class DataTransfer:
    def __init__(self):
        # Lấy thông tin kết nối từ biến môi trường
        self.user = 'root'
        self.password = os.getenv('DB_PASSWORD')
        self.host = 'localhost'
        self.database = 'tiki'
        self.mongodb_name = 'ecommerce'
        self.mongo_client = None
        # Kiểm tra xem các biến môi trường đã được thiết lập hay chưa
        if not all([self.user, self.password, self.host, self.database]):
            raise ValueError("Các biến môi trường không được thiết lập đầy đủ.")

    def mongodb_connection(self):
        # Kiểm tra kết nối tới MongoDB
        try:
            self.mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
            collection_names = self.mongo_client.list_database_names()
            if self.mongodb_name not in collection_names:
                raise ValueError(f"Collection {self.database} không tồn tại trong MongoDB.")
            database = self.mongo_client[self.mongodb_name]
            self.mongo_collection = database['products']
            print(f"Mongodb connected")
        except pymongo.errors.ConnectionFailure:
            raise ValueError("Không thể kết nối tới MongoDB.")
            
            
            
    def mysql_connection(self):
        # Kiểm tra kết nối tới MySQL
        try:
            self.mysql_cnx = mysql.connector.connect(user=self.user, password=self.password,
                                                     host=self.host)
            self.mysql_cursor = self.mysql_cnx.cursor()
        except mysql.connector.Error:
            raise ValueError("Không thể kết nối tới MySQL.")
            
    def check_create_database(self):
        # Kiểm tra xem cơ sở dữ liệu đã tồn tại hay chưa
        check_query = "SHOW DATABASES LIKE %s"
        self.mysql_cursor.execute(check_query, (self.database,))
        result = self.mysql_cursor.fetchone()
        if result:
            print(f"Cơ sở dữ liệu '{self.database}' đã tồn tại.")
        else:
            # Tạo cơ sở dữ liệu
            create_query = f"CREATE DATABASE {self.database}"
            self.mysql_cursor.execute(create_query)
            print(f"Cơ sở dữ liệu '{self.database}' đã được tạo.")
            
    def check_create_table(self, table):
        # Sử dụng cơ sở dữ liệu
        use_query = f"USE {self.database}"
        self.mysql_cursor.execute(use_query)

        # Kiểm tra xem bảng đã tồn tại hay chưa
        check_query = "SHOW TABLES LIKE %s"
        self.mysql_cursor.execute(check_query, (table,))
        result = self.mysql_cursor.fetchone()
        if result:
            print(f"Bảng '{table}' đã tồn tại.")
        else:
            create_query = """
                CREATE TABLE tiki_product_data (
                    id BIGINT PRIMARY KEY,
                    product_name LONGTEXT,
                    short_description LONGTEXT,
                    description LONGTEXT,
                    url LONGTEXT,
                    rating DECIMAL(4, 2),
                    sold_quantity BIGINT,
                    price DECIMAL(30, 2),
                    category_id VARCHAR(100),
                    day_ago_created VARCHAR(100)
                )
            """
            self.mysql_cursor.execute(create_query)
            print(f"Bảng '{table}' đã được tạo.")
            
            
    def extract_data(self, doc):
        id_product = doc.get("id", "")
        print(id_product)
        product_name = doc.get("name", "")
        short_description = doc.get("short_description", "")
        url = doc.get("short_url", "")
        description = doc.get("description", "")
        rating = doc.get("rating_average", "")
        sold_quantity = doc.get("quantity_sold")
        val = 0
        if sold_quantity:
            val = sold_quantity.get("value") or 0
        price = doc.get("price", "")
        category = doc.get("categories", {})
        category_id = 0
        if category:
            category_id = category.get("id", "")
        day_ago_created = doc.get("day_ago_created") or 0
        # Làm sạch mô tả bằng cách loại bỏ các tag HTML thừa
        soup = BeautifulSoup(description, "html.parser")
        cleaned_description = soup.get_text()

        return (id_product, product_name, short_description, url,cleaned_description, rating, val, price, category_id,day_ago_created)
    
    def insert_data_to_mysql(self, table):
        # Kiểm tra và tạo cơ sở dữ liệu (nếu chưa tồn tại)
        self.check_create_database()

        # Kiểm tra và tạo bảng (nếu chưa tồn tại)
        self.check_create_table(table)
        
        # Chèn dữ liệu vào MySQL
        insert_query = f"""
            INSERT INTO {table} (id, product_name, short_description, url, description, rating, sold_quantity, price, category_id, day_ago_created)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s)
        """
#         self.mysql_cnx = mysql.connector.connect(user=self.user, password=self.password,
#                                                      host=self.host)

        # Thực hiện truy vấn aggregate
        pipeline = [
            {
                "$project": {
                    "_id": 0,
                    "id": 1,
                    "name": 1,
                    "short_description": 1,
                    "short_url": 1,
                    "description": 1,
                    "rating_average": 1,
                    "quantity_sold.value": 1,
                    "quantity_sold.price": 1,
                    "categories.id": 1,
                    "day_ago_created": 1
                }
            },
            {
                "$out": "exported_collection"
            }
        ]
        mongo_data = self.mongo_collection.aggregate(pipeline)
        list_data = []
        for doc in mongo_data:
            data = self.extract_data(doc)
            list_data.append(data)
            if len(list_data) == 3000:
                self.mysql_cursor.executemany(insert_query, list_data)
                self.mysql_cnx.commit()
                list_data = []

        # Chèn dữ liệu còn lại vào MySQL
        if len(list_data) > 0:
            self.mysql_cursor.executemany(insert_query, list_data)
            self.mysql_cnx.commit()

        # Lưu thay đổi vào cơ sở dữ liệu MySQL
        self.mysql_cnx.commit()

    def close_connection(self):
        # Đóng con trỏ và kết nối MySQL
        self.mysql_cursor.close()
        self.mysql_cnx.close()

        # Đóng kết nối MongoDB
        self.mongo_client.close()

def main():
    data_transfer = DataTransfer()
    data_transfer.mongodb_connection()
    data_transfer.mysql_connection()
    # Chèn dữ liệu vào MySQL
    data_transfer.insert_data_to_mysql('tiki_product_data')
    # Đóng kết nối
    data_transfer.close_connection()

if __name__ == "__main__":
    main()


Mongodb connected
Cơ sở dữ liệu 'tiki' đã tồn tại.
Bảng 'tiki_product_data' đã tồn tại.
{'_id': ObjectId('649942c8e9c823ca18daca0b'), 'id': 46600215, 'sku': '6636922032221', 'name': 'Hộp kẹp tóc cho bé - Màu cam (Quà tặng đáng yêu cho bé gái) - kèm hộp có quai xách', 'url_key': 'hop-kep-toc-cho-be-mau-cam-qua-tang-dang-yeu-cho-be-gai-kem-hop-co-quai-xach-p46600215', 'url_path': 'hop-kep-toc-cho-be-mau-cam-qua-tang-dang-yeu-cho-be-gai-kem-hop-co-quai-xach-p46600215.html?spid=46600216', 'type': 'simple', 'author_name': '', 'book_cover': None, 'brand_name': 'OEM', 'short_description': '- Sản phẩm hộp kẹp tóc bao gồm: 13 loại kẹp khác nhau: kẹp tóc nơ lớn, nơ nhỏ, kẹp tóc thỏ, kẹp tóc vương miệng, kẹp hoa, kẹp tóc ngôi cột tóc, trong đó là 2 cột đôi 2 bên và 1 dây băng.- Các kẹp tóc ...', 'price': 67000, 'list_price': 67000, 'badges': [{'code': 'new_pdp', 'text': 'v1'}], 'badges_new': [], 'discount': 0, 'discount_rate': 0, 'rating_average': 4.7, 'review_count': 211, 'order_count': 0, 'favo